In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
import os
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.models import load_model
import tensorflow
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
tensorflow.random.set_seed(20)
from keras_tuner.tuners import RandomSearch

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import warnings
warnings.filterwarnings("ignore") 

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

## Read data

In [ ]:
labels=pd.read_excel('data.xlsx')

In [ ]:
#Normalize data
from sklearn.preprocessing import MinMaxScaler
ss=MinMaxScaler()
cols=['u', 'PCF', 'EA', 'SEA', 'k']
tmp=pd.DataFrame(ss.fit_transform(labels[cols]),columns=cols)
for col in cols:
    labels[col]=tmp[col]

In [ ]:
labels

## Create training and testing data – same as Xception

In [ ]:
import os
import shutil
import random

image_folder = "./images"
train_folder = './train'
test_folder = './test'

# if os.path.exists(train_folder):
#     shutil.rmtree(train_folder)
# if os.path.exists(test_folder):
#     shutil.rmtree(test_folder)
# os.makedirs(train_folder, exist_ok=True)
# os.makedirs(test_folder, exist_ok=True)


# image_files = [i for i in os.listdir(image_folder) if i.find('jpg')>=0]
# random.shuffle(image_files)


# total_images = len(image_files)
# train_images = int(total_images * 0.7)
# test_images = total_images - train_images


# for i, image_file in enumerate(image_files):
#     source_path = os.path.join(image_folder, image_file)


#     if i < train_images:
#         destination_folder = train_folder
#     else:
#         destination_folder = test_folder


#     destination_path = os.path.join(destination_folder, image_file)
#     shutil.copyfile(source_path, destination_path)

In [ ]:
train_list=[i for i in os.listdir(train_folder) if i.find('jpg')>=0]
test_list=[i for i in os.listdir(test_folder) if i.find('jpg')>=0]

In [ ]:
train_csv=labels[labels.name.isin(train_list)]
test_csv=labels[labels.name.isin(test_list)]

In [ ]:
label_list=['p','u', 'PCF', 'EA', 'SEA', 'k']

## Load dataset

In [ ]:

from tensorflow.keras.preprocessing.image import load_img, img_to_array


class CustomDataGenerator(Sequence):
    def __init__(self, csv_file, directory, batch_size, target_size, label_list, shuffle=True, augment=False):
        self.csv_file = csv_file
        self.directory = directory
        self.batch_size = batch_size
        self.target_size = target_size
        self.label_list = label_list
        self.shuffle = shuffle
        self.augment = augment
        self.on_epoch_end()
        
        if self.augment:
            self.image_data_generator = ImageDataGenerator(
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest')
        else:
            self.image_data_generator = ImageDataGenerator()

    def __len__(self):
        return int(np.floor(len(self.csv_file) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.csv_file.iloc[k] for k in indexes]

        X, y = self.__data_generation(batch)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.csv_file))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch):
        X1 = np.empty((self.batch_size, *self.target_size, 3))
        X2 = np.empty((self.batch_size, 1))
        y = np.empty((self.batch_size, len(self.label_list) - 1))

        for i, data in enumerate(batch):
            img_path = f"{self.directory}/{data['name']}"
            image = load_img(img_path, target_size=self.target_size)
            # image = img_to_array(image) / 255.0  
            image = img_to_array(image)  

            if self.augment:
                image = self.image_data_generator.random_transform(image)
            
            X1[i, ] = image
            X2[i, ] = data[self.label_list[0]]
            y[i, ] = data[self.label_list[1:]]

        return [X1, X2], y

In [ ]:
batch_size=32
target_size = (256, 256)

train_generator = CustomDataGenerator(train_csv, './train', batch_size, target_size, label_list, shuffle=False)
test_generator = CustomDataGenerator(test_csv, './test', batch_size, target_size, label_list, shuffle=False)

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name() 
physical_devices = tf.config.experimental.list_physical_devices('GPU')  
for device in physical_devices:  
     print(device)

In [ ]:
from tqdm import tqdm
y_train = []

for i in tqdm(range(len(train_generator))):
    batch_data, batch_labels = train_generator[i]
    y_train.append(batch_labels)

y_train = np.concatenate(y_train)
y_test = []

for i in tqdm(range(len(test_generator))):
    batch_data, batch_labels = test_generator[i]
    y_test.append(batch_labels)

y_test = np.concatenate(y_test)

In [ ]:
def plot_model_history(model_history,model_name):
    fig, axs = plt.subplots(1,2,figsize=(12,4),dpi=120)
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['mse'])+1),model_history.history['mse'])
    axs[0].plot(range(1,len(model_history.history['val_mse'])+1),model_history.history['val_mse'])
    axs[0].set_title('Model mse')
    axs[0].set_ylabel('mse')
    axs[0].set_xlabel('Epoch')
    # axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    # axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    # axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    fig.savefig(model_name+'_loss.jpg',dpi=600)
    plt.show()

In [ ]:
def build_model(hp):
    input_image = Input(shape=(224, 224, 3))
    input_features = Input(shape=(1,))

    base_model = tf.keras.applications.vgg16.VGG16(
        weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg'
    )

    from tensorflow.keras.applications.vgg16 import preprocess_input
    x_in = tf.keras.layers.Lambda(preprocess_input)(input_image)
    x = base_model(x_in)                 # [batch, 512]
    x = Concatenate()([x, input_features])

    activations = ['relu', 'tanh', 'relu']
    for i, act in enumerate(activations):
        units = hp.Int(f'units_{i}', min_value=64, max_value=512, step=64)
        x = Dense(units=units, activation=act)(x)

    output = Dense(5)(x)
    model = Model(inputs=[input_image, input_features], outputs=output)
    lr = hp.Float('lr', min_value=3e-5, max_value=1e-3, sampling='LOG')
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse', metrics=['mse'])
    return model


tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=100,
    executions_per_trial=1,
    directory='my_dir',
    project_name='vgg16_tuning'
)

tuner.search_space_summary()

tuner.search(train_generator, epochs=10, validation_data=test_generator)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Learning rate: {best_hps.get('lr')}")

activations = ['relu', 'tanh', 'relu']
for i, act in enumerate(activations):
    print(f"Layer {i+1}: units={best_hps.get(f'units_{i}')}, activation={act}")



In [ ]:

model = tuner.hypermodel.build(best_hps)

# checkpoint = tf.keras.callbacks.ModelCheckpoint("./model.ckpt", monitor='val_loss', verbose=1,
#                                                 save_best_only=True, mode='min', save_weights_only=True)

# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "best_model.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only=False
)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)



In [ ]:
model_info = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=100,
    validation_data=test_generator,
    validation_steps=len(test_generator),
    callbacks=[checkpoint, early_stopping]
)

In [ ]:

import joblib
joblib.dump(ss, 'scaler.pkl')  
print("Model and scaler saved successfully!")

In [ ]:

# loss_values = model_info.history['loss']
# val_loss_values = model_info.history['val_loss']
# df_loss = pd.DataFrame({'epoch': np.arange(1, num_epoch + 1), 'loss': loss_values, 'val_loss': val_loss_values})
# df_loss.to_csv('loss_curve.csv', index=False)

In [ ]:
plot_model_history(model_info, 'Xception')

## Model evaluation

In [ ]:

# model.load_weights("./model.ckpt")
from tensorflow.keras.models import load_model
model = load_model('./best_model.h5')

In [ ]:
from sklearn import metrics
y_test_pred=model.predict(test_generator)
y_train_pred=model.predict(train_generator)

In [ ]:

y_test_pred=ss.inverse_transform(y_test_pred)
y_test_true=ss.inverse_transform(y_test)

In [ ]:

y_train_pred=ss.inverse_transform(y_train_pred)
y_train_true=ss.inverse_transform(y_train)

In [ ]:
# Define model evaluation metrics
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error 
from sklearn import metrics
def model_result(y_train,y_predict1,y_test,y_predict):
    print('train_index：')
    print('MSE: ', '%.4f'%float(mean_squared_error(y_train,y_predict1)),\
    'MAE: ', '%.4f'%float(mean_absolute_error(y_train,y_predict1)),\
    'RMSE:',' %.4f'%float(np.sqrt(mean_squared_error(y_train, y_predict1))),\
    'R2: ', '%.4f'%float(metrics.r2_score(y_train, y_predict1)))
    
    print('test_index')
    print('MSE: ', '%.4f'%float(mean_squared_error(y_test,y_predict)),\
    'MAE: ', '%.4f'%float(mean_absolute_error(y_test,y_predict)),\
    'RMSE:', '%.4f'%float(np.sqrt(mean_squared_error(y_test, y_predict))),\
    'R2: ', '%.4f'%float(metrics.r2_score(y_test, y_predict)))
    print('\n')
label_list2=['u', 'PCF', 'EA', 'SEA', 'k']
for i in range(1,len(label_list2)+1):
    print(label_list2[i-1])
    model_result(y_train_true[:,i-1].ravel(),y_train_pred[:,i-1].ravel(),y_test_true[:,i-1].ravel(),y_test_pred[:,i-1].ravel())

In [ ]:
import matplotlib
# matplotlib.rcParams['font.family']='SimHei''STSong'
matplotlib.rcParams['axes.unicode_minus']=False
color_list=['red','blue','purple','orange','yellow','green','pink','aquamarine','dodgerblue','purple','red','blue','orange','yellow','green']
plt.figure(figsize=(12,10),dpi=120)
for i in range(1,len(label_list2)+1):
    plt.subplot(3,3,i)
    plt.scatter(y_test_pred[:,i-1].ravel(),y_test_true[:,i-1].ravel(),color=color_list[i-1])
    plt.title(label_list2[i-1])
    plt.xlabel('Predict')
    plt.ylabel('True')
    plt.tight_layout()
plt.savefig('compaired.jpg',dpi=600,bbox_inches = 'tight')
plt.show()

In [ ]:
# Export true–predicted relationship data to Excel file
with pd.ExcelWriter('true_vs_predicted_comparison.xlsx') as writer:
    for i in range(1, 6):
        df_comparison = pd.DataFrame({
            'True_Value': y_test_true[:, i - 1].ravel(),
            'Predicted_Value': y_test_pred[:, i - 1].ravel()
        })
        df_comparison.to_excel(writer, sheet_name=label_list2[i - 1], index=False)